<a href="https://colab.research.google.com/github/nhso/eng-nsho-scraping/blob/main/%E0%B8%AA%E0%B8%9B%E0%B8%AA%E0%B8%8A_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup


In [3]:
url = "http://eng.nhso.go.th/view/1/Philosophy_Background/EN-US"
data = requests.get(url)

soup = BeautifulSoup(data.text,'html.parser')
Philosophy_Background = soup.find_all("div",{"id":"content-B"})

for i in Philosophy_Background:
  print(i.text)



Philosophy & Background


Philosophy 


“Health” is considered as an important component of the “quality of life”


“Health security” is a system of life security which gives a person confidence that they will be able to indulge in appropriate health behavior, and would be able to get complete access to health promotion and disease prevention services. They should also be able to access the quality health services without any obstacles when they are ill and in need of medical care.


“Equity” of health can take place only when those with different social, economical and cultural status have the equal opportunity to get access to quality health care, whereby their responsibility to pay for the expenses should depend on their ability to pay.


“Universal” indicates the basic philosophy under the national health security for which all people should have the right to obtain universal access. For whatever reason, nobody should be left out, oven on one’s own, based on equal respect of the 

In [4]:
url = "http://eng.nhso.go.th/view/1/Vision_Mission/EN-US"
data = requests.get(url)

soup = BeautifulSoup(data.text,'html.parser')
Vision_Mission = soup.find_all("div",{"id":"content-A"})

for i in Vision_Mission:
  print(i.text)




HomeAbout_NHSOVision/Mission

Vision


Everyone who lives in Thailand is covered by UHC and access to health care with confidence when needed
 


Mission


Secure people toward effective equitable responsive coverage, access, and utilization by evidence-informed decision and participation
 


Goals


3 Goals of “CSG”


Effective, Equitable and Responsive Coverage


SAFE Financing System


Good Governance





Strategies


5 Ensure Strategies 
Strategy 1 : Ensure coverage and access for vulnerable and underutilization groups
Strategy 2 : Ensure quality and adequacy of health services
Strategy 3 : Ensure financial efficiency
Strategy 4 : Ensure participation and ownership of all stakeholders
Strategy 5 : Ensure good governance 




In [5]:
url = "http://eng.nhso.go.th/view/1/NHSO_Board/EN-US"
data = requests.get(url)

soup = BeautifulSoup(data.text,'html.parser')
NHSO_Board = soup.find_all("div",{"id":"content-A"})

for i in NHSO_Board:
  print(i.text)




HomeAbout_NHSONHSO_StructureNHSO BoardNHSO’s Board


National Health Security  Board


Standard and Quality Control Board



National Health Security Board
The National Health Security Act. 2002. Section 13  “National Health Security Board” made up of:
 
1. The Minister of Public Health as a Chairman,
2. The Permanent Secretary for Defense, Permanent Secretary for Finance, Permanent Secretary for Commerce, Permanent Secretary for the Interior, Permanent Secretary for Labor and Social Welfare, Permanent Secretary for Public Health, Permanent Secretary for University Affairs, and the Director of the Office of the Budget, 
3. A representative of each Municipality, a representative of each local Provincial Administrative Organization, a representative of each local Tambon Administrative Organization, and a representative of other local government organizations elected by executives of its organization, 
4. Five representatives of, elected by, representatives each of which from a non-pro

In [6]:
url = "http://eng.nhso.go.th/view/1/NHSO_Executive/EN-US"
data = requests.get(url)

soup = BeautifulSoup(data.text,'html.parser')
NHSO_Executive = soup.find_all("div",{"id":"content-A"})

for i in NHSO_Executive:
  print(i.text)




HomeAbout_NHSONHSO_StructureNHSO Executive



Secretary-General
Dr. Jadej Thammatacharee
jadej.t@nhso.go.th








Deputy Secretary-General
Dr. Jakkrit Ngowsiri
jakkrit.n@nhso.go.th






Deputy Secretary-General
Dr. Athaporn Limpanylers
atthaporn.l@nhso.go.th






Deputy Secretary-General
Dr. Sinchai Tawwuttanakidgul
sinchai.t@nhso.go.th








Deputy Secretary-General
Dr. Lalitaya Kongkam
lalitaya.k@nhso.go.th






Deputy Secretary-General
Dr. Aphichat Rodsom
aphichat.r@nhso.go.th






Deputy Secretary-General
Dr. Yupadee Sirisinsuk
yupadee.s@nhso.go.th








Assistant Secretary-General
Mrs.Waraporn Suwanwela
waraporn.s@nhso.go.th






Assistant Secretary-General
Mrs.Benjamas Lerdchakorn
benjamas.l@nhso.go.th



 
4 FEB 2021







In [7]:
url = "http://eng.nhso.go.th/view/1/NHSO_Regional_Office/EN-US"
data = requests.get(url)

soup = BeautifulSoup(data.text,'html.parser')
NHSO_Regional_Office = soup.find_all("div",{"id":"content-B"})

for i in NHSO_Regional_Office:
  print(i.text)

 Head quarter & Regional Office Address


 











Branch


Address


Telephone


Fax




Head quarter, Bangkok  


National Health Security Office
The Government Complex Commemoration, His Majesty the King's 80th Birthday Anniversary 5th December, Building B, 2-4 Fl.,  120 Moo 3, Chaengwattana Rd., Lak Si District, Bangkok 10210 Thailand


(662)1414000


(662)1439731




National Health Security Office Region 1 Chiang Mai


6 Post Office Region 5, 2nd Fl., 6 Mahidol Rd., Muang , Chiang Mai 50200 Thailand


(6653)285355


(6653)285364




National Health Security Office Region 2 Phitsanulok


118 Phitsanulok Post Office Building, 4th Fl., 
Bhuttabucha Rd., Naimuang, Muang, Phitsanulok 65000 Thailand


(6655)245111


(6655)245111




National Health Security Office Region 3 Nakhonsawan


1045/2, Moo 10 Nakhonsawan-Phitsanulok Rd., Nakhonsawantok , Muang, Nakhonsawan 60000 Thailand


(6656)371831-7


(6656)371838




National Health Security Office Region 4 Saraburi


65/3 Soi 1, Pic

In [8]:
url = "http://eng.nhso.go.th/view/1/NHSB/EN-US"
data = requests.get(url)

soup = BeautifulSoup(data.text,'html.parser')
NHSB = soup.find_all("div",{"id":"content-A"})

for i in NHSB:
  print(i.text)




HomeAbout_NHSONHSO_StructureNational Health Security Board



Minister of Public Health
Mr. Anutin Charnvirakul
Chairperson








Permanent Secretary of Defence
Gen. Natt Intracharoen
Committee






Permanent Secretary of Finance
Mr. Prasong Poontaneat
Committee






Permanent Secretary of Commerce
Mr. Boonyarit Kalayanamit
Committee






Permanent Secretary of Interior
Mr. Chatchai Promlert
Committee






Permanent Secretary of Labour
Mr. Suthi Sukosol
Committee






Permanent Secretary of Education
Mr. Prasert Boonruang
Committee






Permanent Secretary of Public Health
Dr. Kiattiphum Wongrajit
Committee






Director of the Budget Bureau
Mr. Dechapiwat Na Songkhla
Committee






Representative of the Municipalities
 
Due Consideration
Committee






Representative of the Provincial Administrative Organization
Mr. Anuvart Wongwan
Committee






Representative of the Sub-District Administrative Organization
Mr. Pramot Petcharat
Committee






Representative of other L

In [9]:
url = "http://eng.nhso.go.th/view/1/QSC/EN-US"
data = requests.get(url)

soup = BeautifulSoup(data.text,'html.parser')
QSC = soup.find_all("div",{"id":"content-A"})

for i in QSC:
  print(i.text)




HomeAbout_NHSONHSO_StructureQuality and Standard Control Board



Chairperson of the Quality and Standard Control Board
Dr. Suphan Srithamma








Advisor of the Quality and Standard Control Board
Dr. Chatri Banchuin






Advisor of the Quality and Standard Control Board
Mr. Jon Ungphakorn






Advisor of the Quality and Standard Control Board
Dr. Panya Keeratihuntayakorn






Director-General of the Department of Medical Services
Dr. Tanin Sonthirak






Secretary-General of the Food and Drug Administration
Dr. Paisarn Dunkum






President of the Hospital Development and Accreditation Institute
Dr. Theerapol Topanthanont






Director of the Bureau of Sanatorium and Art of Healing
Dr. Nalina Tantiniramai






Representative of Medical Council
Dr. Pinit Hirunyachote






Representative of Nursing and Midwifery Council
 
Assoc. Prof. Dr. Tassana Boontong






Representative of Dental Council
Dr. Santiphap Meesawat






Representative of Pharmacy Council
Pharm. Jira Wipha

In [10]:
url = "http://eng.nhso.go.th/view/1/National_Health_Security_Act_B.E.2545/EN-US"
data = requests.get(url)

soup = BeautifulSoup(data.text,'html.parser')
National_Health_Security_Act_B_E_2545 = soup.find_all("div",{"id":"content-B"})

for i in National_Health_Security_Act_B_E_2545:
  print(i.text)



National Health Security Act B.E.2545 (A.D.2002)
National Health Security Act B.E. 2545 (A.D. 2002) His Majesty King Bhumibol Adulyadej is graciously pleased to proclaim that whereas it is expedient to have the law on national health security. Given on the 11th Day of November B.E. 2545.
 
The National Health Security Act was enacted and published in the Government Gazette on 18th November B.E. 2545 (2002) and coming into force on the next day, 19th November B.E. 2545 (2002). The main features of Act are as shown in following table
 
Main features of the National Health Security Act, B.E.2545 (2002)
 
 









Feature


National Health Security Act




1. Definition of health Services


Services for disease prevention, disease diagnosis, medical treatment , health promotion, and rehabilitation, including Thai traditional and alternative medicine service




2. Right to receive health services


Every person has the right to receive that services that are of good standard and in an 

In [11]:
url = "http://eng.nhso.go.th/view/1/Contact_NHSO/EN-US"
data = requests.get(url)

soup = BeautifulSoup(data.text,'html.parser')
Contact_NHSO = soup.find_all("div",{"id":"content-B"})

for i in Contact_NHSO:
  print(i.text)










National Health Security Office



The Government Complex Commemorating His Majesty the King's 80th Birthday Anniversary 5th December, B.E.2550 (2007) Building B 120 Moo 3 Chaengwattana Road, Lak Si District, Bangkok 10210

Phone  :  02-141-4000 (Office hours)
Fax  :  02-143-9730 - 1
Website  :  http://eng.nhso.go.th/
GPS  :  13.8828179, 100.5652935
E-mail  :  Internhso@nhso.go.th
Facebook  :  https://ww.facebook.com/NHSOInter 




 






